In [ ]:
import os
GRID_SEARCH_DIR = "intervention_logs"

for file in os.listdir(GRID_SEARCH_DIR):
    print(file)

In [ ]:
import pandas as pd
import os
import numpy as np
import json
import re

GRID_SEARCH_DIR = "intervention_logs"

def extract_info_from_filename(filename: str) -> dict:
    """
    Extracts information from the filename using regular expressions.

    Parameters:
    filename (str): The filename from which to extract the information.

    Returns:
    dict: A dictionary containing the extracted information.
    """
    # Define the regex pattern to extract the needed information
    pattern = r"sampling=both_intervention_type=(?P<intervention_type>\w+)_first_layer=(?P<first_layer>\d+)_last_layer=(?P<last_layer>\d+)_p=(?P<p>\d+\.\d+)_b=(?P<b>\d+\.\d+)_scales="
    
    # Use the regex search function to find matches
    match = re.search(pattern, filename)

    # If a match is found, extract the information
    if match:
        info = match.groupdict()
        # Convert numeric values from strings to their appropriate types
        info['first_layer'] = int(info['first_layer'])
        info['last_layer'] = int(info['last_layer'])
        info['p'] = float(info['p'])
        info['b'] = float(info['b'])
        return info
    else:
        return {"error": "No match found"}

# List all CSV files
json_files = [f for f in os.listdir(GRID_SEARCH_DIR) if f.endswith('.json')]

# Dictionary to hold file names and their average scores
average_scores_dict = {}

for file in json_files:
    file_info = extract_info_from_filename(file)
    if "error" in file_info:
        continue
    # if file_info["first_layer"] != file_info["last_layer"]:
    #     continue

    with open(f"{GRID_SEARCH_DIR}/{file}", "r") as f:
        data = json.load(f)
    if "possible_sampled_moves" not in data:
        continue
    average_scores_dict[file] = {}
    average_scores_dict[file]["possible_sampled_moves"] = data["possible_sampled_moves"]
    for key in data:
        try:
            # Try to convert the key to float
            float_key = float(key)
            # If conversion is successful, add the key to the dictionary
            average_scores_dict[file][key] = data[key]
        except ValueError:
            # If conversion fails, the key is not a float
            pass

sampled_ratio_list = []
for file in average_scores_dict:
    max_possible = average_scores_dict[file]["possible_sampled_moves"]
    for scale in average_scores_dict[file]:
        if scale == "possible_sampled_moves":
            continue
        legal_sampled = average_scores_dict[file][scale]["mod_board_sampled_legal_total"]
        sampled_ratio = legal_sampled / max_possible
        average_scores_dict[file][scale]["sampled_ratio"] = sampled_ratio
        sampled_ratio_list.append((file, scale, sampled_ratio))




# Sort the files by average score
sorted_sampled_ratio_list = sorted(sampled_ratio_list, key=lambda x: x[2], reverse=True)

# Print sorted files and their scores
for file, scale, score in sorted_sampled_ratio_list:
    # if "gradient" not in file:
    #     continue
    # if scale != "3.0":
    #     continue
    print(f"{file}, {scale}, {score}")


In [ ]:
import pandas as pd
import os
import numpy as np
import json
import re

GRID_SEARCH_DIR = "intervention_logs"

def extract_info_from_filename(filename: str) -> dict:
    """
    Extracts information from the filename using regular expressions.

    Parameters:
    filename (str): The filename from which to extract the information.

    Returns:
    dict: A dictionary containing the extracted information.
    """
    # Define the regex pattern to extract the needed information
    pattern = r"sampling=both_n_layers=8_intervention_type=(?P<intervention_type>\w+)_first_layer=(?P<first_layer>\d+)_last_layer=(?P<last_layer>\d+)_p=(?P<p>\d+\.\d+)_b=(?P<b>\d+\.\d+)_iters=(?P<iters>\d+)_scales="
    
    # Use the regex search function to find matches
    match = re.search(pattern, filename)

    # If a match is found, extract the information
    if match:
        info = match.groupdict()
        # Convert numeric values from strings to their appropriate types
        info['first_layer'] = int(info['first_layer'])
        info['last_layer'] = int(info['last_layer'])
        info['p'] = float(info['p'])
        info['b'] = float(info['b'])
        return info
    else:
        return {"error": "No match found"}

# List all CSV files
json_files = [f for f in os.listdir(GRID_SEARCH_DIR) if f.endswith('.json')]

# Dictionary to hold file names and their average scores
average_scores_dict = {}

for file in json_files:
    file_info = extract_info_from_filename(file)
    if "error" in file_info:
        continue
    # if file_info["first_layer"] != file_info["last_layer"]:
    #     continue

    with open(f"{GRID_SEARCH_DIR}/{file}", "r") as f:
        data = json.load(f)
    if "possible_sampled_moves" not in data:
        continue
    average_scores_dict[file] = {}
    average_scores_dict[file]["possible_sampled_moves"] = data["possible_sampled_moves"]
    for key in data:
        try:
            # Try to convert the key to float
            float_key = float(key)
            # If conversion is successful, add the key to the dictionary
            average_scores_dict[file][key] = data[key]
        except ValueError:
            # If conversion fails, the key is not a float
            pass

sampled_ratio_list = []
for file in average_scores_dict:
    max_possible = average_scores_dict[file]["possible_sampled_moves"]
    for scale in average_scores_dict[file]:
        if scale == "possible_sampled_moves":
            continue
        legal_sampled = average_scores_dict[file][scale]["mod_board_sampled_legal_total"]
        sampled_ratio = legal_sampled / max_possible
        average_scores_dict[file][scale]["sampled_ratio"] = sampled_ratio
        sampled_ratio_list.append((file, scale, sampled_ratio))




# Sort the files by average score
sorted_sampled_ratio_list = sorted(sampled_ratio_list, key=lambda x: x[2], reverse=True)

# Print sorted files and their scores
for file, scale, score in sorted_sampled_ratio_list:
    # if "gradient" not in file:
    #     continue
    # if scale != "3.0":
    #     continue
    print(f"{file}, {scale}, {score}")


In [ ]:
import pandas as pd
import os
import numpy as np
from scipy.stats import sem, t

def compute_average_score_and_confidence_interval(file_path: str, player: str = "player_one", confidence=0.95) -> dict:
    file_path = os.path.join(GRID_SEARCH_DIR, file_path)
    df = pd.read_csv(file_path)

    # Filter rows where 'Stockfish -1' is in 'game_title'
    # df = df[df['game_title'].str.contains("Stockfish 0")]
    df = df[~df['game_title'].str.contains("Stockfish -1")]

    # df = df[df['player_one_failed_to_find_legal_move'] == False]

    total_moves = len(df)
    successful_moves = df[df[f'{player}_failed_to_find_legal_move'] == False].shape[0]
    percentage_successful = (successful_moves / total_moves) * 100

    df[f"{player}_score"] = pd.to_numeric(df[f"{player}_score"], errors="coerce")

    # Compute overall average score and confidence interval
    scores = df[f"{player}_score"].dropna()
    if len(scores) > 1:
        mean_score = np.mean(scores)
        std_err = sem(scores)
        h = std_err * t.ppf((1 + confidence) / 2, len(scores) - 1)
        result = {"mean_score": mean_score, "confidence_interval": (mean_score - h, mean_score + h)}
    else:
        result = {"mean_score": scores.iloc[0], "confidence_interval": (np.nan, np.nan)}

    result["percentage_successful"] = percentage_successful
    result["total_moves"] = total_moves

    # print(result)

    return result

    # # Compute average scores and confidence intervals
    # results = {}
    # for game_title, group in df.groupby("game_title"):
    #     scores = group[f"{player}_score"].dropna()
    #     if len(scores) > 1:
    #         mean_score = np.mean(scores)
    #         std_err = sem(scores)
    #         h = std_err * t.ppf((1 + confidence) / 2, len(scores) - 1)
    #         results[game_title] = {"mean_score": mean_score, "confidence_interval": (mean_score - h, mean_score + h)}
    #     else:
    #         results[game_title] = {"mean_score": scores.iloc[0], "confidence_interval": (np.nan, np.nan)}

    # print(results)
    # return results

# List all CSV files
json_files = [f for f in os.listdir(GRID_SEARCH_DIR) if f.endswith('.csv')]

# Dictionary to hold file names and their average scores
results_dict = {}
average_scores_dict = {}

for file in json_files:
    try:
        # if "10_random_moves" not in file:
        #     continue
        # if "0_1_coefficient" in file:
        #     continue
        if "levels_14" in file or "levels_15" in file or "levels_04" in file:
            continue
        if "200k" in file:
            continue
        if "pos_start_32" in file:
            continue
        if "20000_moves" in file:
            continue
        # if "layer_8" in file:
        #     continue
        # if "8layers" not in file and "layers_8" not in file:
        if "8layers" in file or "layers_8" in file:
            continue
        result = compute_average_score_and_confidence_interval(file)
        average_scores_dict[file] = result["mean_score"]
        results_dict[file] = result
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Sort the files by average score
sorted_files = sorted(average_scores_dict.items(), key=lambda x: x[1], reverse=True)

# Print sorted files and their scores
for file, score in sorted_files:
    if "10_random_moves" not in file:
        continue
    confidence_interval = results_dict[file]["confidence_interval"]
    total_moves = results_dict[file]["total_moves"]
    percentage_successful = results_dict[file]["percentage_successful"]
    # if file.startswith("lichess_train"):
    #     file = file.replace("lichess_train", "lichess_000k_bins_train")
    # if total_moves <= 100:
    #     continue

    print(f"{file}: {score}, {total_moves}, {percentage_successful}, {confidence_interval}")

print("\n\nBegin 0 random moves\n\n")
# Print sorted files and their scores
for file, score in sorted_files:
    if "_0_random_moves" not in file:
        continue
    confidence_interval = results_dict[file]["confidence_interval"]
    total_moves = results_dict[file]["total_moves"]
    percentage_successful = results_dict[file]["percentage_successful"]
    # if file.startswith("lichess_train"):
    #     file = file.replace("lichess_train", "lichess_000k_bins_train")
    # if total_moves <= 100:
    #     continue

    print(f"{file}: {score}, {total_moves}, {percentage_successful}, {confidence_interval}")
